In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from constants import LABEL_COLS

In [2]:
f1_scores = []
for cutoff in np.linspace(0, 1, num=50):
    results_df = pd.read_csv('../data/predicted_labels.csv')

    binary_labels = []
    for key in LABEL_COLS:
        binary_key = f'binary_{key}'
        results_df[binary_key] = (results_df[key] >= cutoff).astype(int)

        binary_labels.append(binary_key)
    results_df.drop(LABEL_COLS, axis=1, inplace=True)
    data_df = pd.read_csv('../data/raw/test.csv')

    label_df = pd.read_csv('../data/raw/test_labels.csv')

    test_df = data_df.set_index('id').join(label_df.set_index('id'))
    master_df = test_df.join(results_df.set_index('id'))
    CONDITIONS = [f'{label} != -1' for label in LABEL_COLS]
    QUERY_STRING = ' & '.join(CONDITIONS)
    master_df = master_df.query(QUERY_STRING)
    y_test = master_df[LABEL_COLS]
    predictions = master_df[binary_labels]

    f1_scores.append((cutoff, f1_score(y_test, predictions, average='weighted')))

/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
cutoff, _ = max(f1_scores, key=lambda x: x[1])
results_df = pd.read_csv('../data/predicted_labels.csv')

binary_labels = []
for key in LABEL_COLS:
    binary_key = f'binary_{key}'
    results_df[binary_key] = (results_df[key] >= cutoff).astype(int)

    binary_labels.append(binary_key)
results_df.drop(LABEL_COLS, axis=1, inplace=True)
data_df = pd.read_csv('../data/raw/test.csv')

label_df = pd.read_csv('../data/raw/test_labels.csv')

test_df = data_df.set_index('id').join(label_df.set_index('id'))
master_df = test_df.join(results_df.set_index('id'))
CONDITIONS = [f'{label} != -1' for label in LABEL_COLS]
QUERY_STRING = ' & '.join(CONDITIONS)
master_df = master_df.query(QUERY_STRING)
y_test = master_df[LABEL_COLS]
predictions = master_df[binary_labels]

print(classification_report(y_test, predictions, target_names=LABEL_COLS))

               precision    recall  f1-score   support

        toxic       0.55      0.84      0.67      6090
 severe_toxic       0.43      0.05      0.09       367
      obscene       0.63      0.76      0.69      3691
       threat       0.00      0.00      0.00       211
       insult       0.66      0.67      0.66      3427
identity_hate       0.74      0.36      0.48       712

    micro avg       0.60      0.72      0.65     14498
    macro avg       0.50      0.44      0.43     14498
 weighted avg       0.60      0.72      0.64     14498
  samples avg       0.07      0.07      0.07     14498



/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/quontas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
